# FashionFlow Network Design

FashionFlow is a clothing retailer that needs to optimize its distribution network. They have:
- 5 manufacturing facilities
- 2 cross-dock facilities
- 5 regional distribution centers
- 2 product lines: Premium and Casual

The goal is to minimize total transportation costs while meeting demand and respecting capacity constraints.

In [ ]:
import pandas as pd
import pulp

In [ ]:
# Inbound transportation costs (Factory to Cross-dock)
inbound_dict = {'Premium':{'Factory 1':{'Crossdock 1':28, 'Crossdock 2':45} ,
                'Factory 2':{'Crossdock 1':25, 'Crossdock 2':60},
                'Factory 3':{'Crossdock 1':32, 'Crossdock 2':15} ,
                'Factory 4':{'Crossdock 1':65, 'Crossdock 2':14} ,
                'Factory 5':{'Crossdock 1':58, 'Crossdock 2':62}},
                'Casual':{'Factory 1':{'Crossdock 1':30, 'Crossdock 2':50} ,
                'Factory 2':{'Crossdock 1':22, 'Crossdock 2':68},
                'Factory 3':{'Crossdock 1':35, 'Crossdock 2':18} ,
                'Factory 4':{'Crossdock 1':70, 'Crossdock 2':15} ,
                'Factory 5':{'Crossdock 1':15, 'Crossdock 2':16}}
}

In [ ]:
# Outbound transportation costs (Cross-dock to DC)
outbound_dict = {'Premium':{'Crossdock 1':{'DC1':15, 'DC2':22, 'DC3':25, 'DC4':38, 'DC5':40} ,
                'Crossdock 2':{'DC1':60, 'DC2':25, 'DC3':20, 'DC4':15, 'DC5':18} } ,
                'Casual':{'Crossdock 1':{'DC1':12, 'DC2':25, 'DC3':28, 'DC4':42, 'DC5':44} ,
                'Crossdock 2':{'DC1':65, 'DC2':28, 'DC3':22, 'DC4':16, 'DC5':20} } ,
}

In [ ]:
# Demand at each DC
demand_dict = {'Premium':{'DC1':120, 'DC2':50, 'DC3':65, 'DC4':90, 'DC5':10} ,
                'Casual':{'DC1':20, 'DC2':40, 'DC3':45, 'DC4':95, 'DC5':165}
}

In [ ]:
# Factory capacity constraints
capacity_dict = {'Premium':{'Factory 1':140, 'Factory 2': 280, 'Factory 3':85, 'Factory 4':130, 'Factory 5':200} ,
            'Casual':{'Factory 1':180, 'Factory 2': 280, 'Factory 3':75, 'Factory 4':40, 'Factory 5':200} ,
            'Combined':{'Factory 1':180, 'Factory 2': 280, 'Factory 3':95, 'Factory 4':140, 'Factory 5':200}
}

In [ ]:
# Create lists of locations and product types
factories = ['Factory 1', 'Factory 2', 'Factory 3', 'Factory 4', 'Factory 5']
CDs = ['Crossdock 1', 'Crossdock 2']
DCs = ['DC1', 'DC2', 'DC3', 'DC4', 'DC5']
styles = ['Premium', 'Casual']

In [ ]:
# Create optimization model
model = pulp.LpProblem("FashionFlow_Network", pulp.LpMinimize)

# Decision variables
# Factory to Cross-dock flow
factory_to_cd = pulp.LpVariable.dicts("factory_to_cd",
                                      ((s, f, cd) for s in styles
                                       for f in factories
                                       for cd in CDs),
                                      lowBound=0)

# Cross-dock to DC flow
cd_to_dc = pulp.LpVariable.dicts("cd_to_dc",
                                 ((s, cd, dc) for s in styles
                                  for cd in CDs
                                  for dc in DCs),
                                 lowBound=0)

In [ ]:
# Objective function
model += (
    pulp.lpSum([factory_to_cd[s,f,cd] * inbound_dict[s][f][cd]
                for s in styles
                for f in factories
                for cd in CDs]) +
    pulp.lpSum([cd_to_dc[s,cd,dc] * outbound_dict[s][cd][dc]
                for s in styles
                for cd in CDs
                for dc in DCs])
)

In [ ]:
# Constraints

# 1. Flow balance at cross-docks
for s in styles:
    for cd in CDs:
        model += pulp.lpSum([factory_to_cd[s,f,cd] for f in factories]) == \
                pulp.lpSum([cd_to_dc[s,cd,dc] for dc in DCs])

# 2. Meet demand at DCs
for s in styles:
    for dc in DCs:
        model += pulp.lpSum([cd_to_dc[s,cd,dc] for cd in CDs]) == demand_dict[s][dc]

# 3. Factory capacity constraints
for f in factories:
    model += pulp.lpSum([factory_to_cd[s,f,cd] 
                         for s in styles
                         for cd in CDs]) <= capacity_dict['Combined'][f]

In [ ]:
# Solve the model
model.solve()
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Total Cost: ${pulp.value(model.objective):,.2f}")

In [ ]:
# Display results
print("\nFactory to Cross-dock Flows:")
for s in styles:
    print(f"\n{s}:")
    for f in factories:
        for cd in CDs:
            if pulp.value(factory_to_cd[s,f,cd]) > 0:
                print(f"{f} -> {cd}: {pulp.value(factory_to_cd[s,f,cd]):,.0f} units")

print("\nCross-dock to DC Flows:")
for s in styles:
    print(f"\n{s}:")
    for cd in CDs:
        for dc in DCs:
            if pulp.value(cd_to_dc[s,cd,dc]) > 0:
                print(f"{cd} -> {dc}: {pulp.value(cd_to_dc[s,cd,dc]):,.0f} units")